# Module 22: Big Data Challenge - Amazon Reviews
## Digital Video Download Review

In [ ]:
import os

# Using Spark Version 3.2.2. To find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [ ]:
#Connection to Postgres Database

!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-23 22:55:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.76MB/s    in 0.2s    

2022-11-23 22:55:00 (5.76 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start Spark session with Postgres connection

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataPart1-DigitalVideo").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


## Extract

1. Connect to and load data from AWS to dataframe using PySpark
2. Include the headers from the dataset
3. Verify data and count of entries

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

# Modified since this is a tab delimited file.
reviewDV_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy") 

# Show Top entries of the DataFrame to be later dissected into the table schemas
reviewDV_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [ ]:
# State the size of the reviewDV_df before dropping NAs
print(reviewDV_df.count())

4057147


## Transform

1. Remove NA from Dataframe
2. Remove duplicate entries
3. Rename columns (as needed) - Not Needed
4. Create separate dataframe for each table

In [ ]:
#Cleaning up the data set to drop all NAs and Duplicate values

# Drop NA values
dropna_df = reviewDV_df.dropna(how='any')

# State the size of the DF after dropping NAs
print(f"Size after dropping NA: {dropna_df.count()}")

# Dropping Duplicate values from the Dataframe
dropna_df = dropna_df.dropDuplicates()

# State the size of the DF after dropping NAs
print(f"Size after removing any duplicates: {dropna_df.count()}")

Size after dropping NA: 4056518
Size after removing any duplicates: 4056518


In [ ]:
# Remove columns not included in the database schema. 
# These columns are not needed: marketplace, product_category, verified_purchase, review_headline, review_body

cleaned_df  = dropna_df.select(["customer_id","review_id","product_id", "product_parent","product_title","star_rating","helpful_votes","total_votes","vine","review_date"])
cleaned_df.show(5)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   51950426|R1005KN8L3OP23|B00COTH9VI|     956367867|Seeking Asian Female|          5|            1|          1|   N| 2015-04-07|
|   42507369|R1008R0427X1FG|B009KHHELW|      41559476|Duck Dynasty Seas...|          4|            0|          0|   N| 2014-05-28|
|    2458036|R100AJRT6FE05K|B0048ZXXIO|     814772102|Team Umizoomi Sea...|          5|            0|          0|   N| 2014-06-27|
|   23459444|R100AOYGH18ZXK|B00GBDWZDU|     936264488|     Christmas Crush|          3|            0|          0|   N| 2015-05-20|
|   38247406|R100BC7LPZKRNN|B007SPQZMC|     192466294|Downton Abbey Sea...|        

In [ ]:
#Examine the dataframe schema to check the datatypes

cleaned_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#Updating the schema to have nullable = False

cleaned_df.schema['customer_id'].nullable=False
cleaned_df.schema['review_id'].nullable=False
cleaned_df.schema['product_id'].nullable=False

print(cleaned_df.schema['customer_id'])
print(cleaned_df.schema['review_id'])
print(cleaned_df.schema['product_id'])

StructField(customer_id,IntegerType,false)
StructField(review_id,StringType,false)
StructField(product_id,StringType,false)


### Create Dataframe for each of the Tables

In [ ]:
# Create review_id dataframe to match review_id_table table
review_id_df = cleaned_df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005KN8L3OP23|   51950426|B00COTH9VI|     956367867| 2015-04-07|
|R1008R0427X1FG|   42507369|B009KHHELW|      41559476| 2014-05-28|
|R100AJRT6FE05K|    2458036|B0048ZXXIO|     814772102| 2014-06-27|
|R100AOYGH18ZXK|   23459444|B00GBDWZDU|     936264488| 2015-05-20|
|R100BC7LPZKRNN|   38247406|B007SPQZMC|     192466294| 2013-03-13|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Create products dataframe to match products table
products_df = cleaned_df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00COTH9VI|Seeking Asian Female|
|B009KHHELW|Duck Dynasty Seas...|
|B0048ZXXIO|Team Umizoomi Sea...|
|B00GBDWZDU|     Christmas Crush|
|B007SPQZMC|Downton Abbey Sea...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
#Create customers dataframe to match customers table and calculate the customer_count
customers_df = cleaned_df.groupby("customer_id").agg({"customer_id": "count"})

# Rename the column to match the table schema
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   36771518|             5|
|   23006345|             5|
|    8899358|             1|
|   45518338|             7|
|   31452416|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
# Create vine dataframe to match vine_table table
vine_df = cleaned_df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005KN8L3OP23|          5|            1|          1|   N|
|R1008R0427X1FG|          4|            0|          0|   N|
|R100AJRT6FE05K|          5|            0|          0|   N|
|R100AOYGH18ZXK|          3|            0|          0|   N|
|R100BC7LPZKRNN|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



## Load

1.   Postgres setup
2.   Write Dataframes to RDS

### Postgres Setup

In [ ]:
# Configure settings for RDS - Please enter your endpoint and database information
mode = "append"
jdbc_url="jdbc:postgresql://<insert endpoint>:5432/<insert database-name>"
config = {"user":"<insert username>", 
          "password": "<insert password>", 
          "driver":"org.postgresql.Driver"}


### Write Dataframes to RDS

In [ ]:
# Write DataFrame to review_id_table table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to active_user table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)